Première cell pour éxécuter dans environnement avec discrete_optimization de github mais pas installer en package python

In [ ]:
import sys 
sys.path.append("/Users/AGAUVZFV/Desktop/discrete-optimization")
import os
os.environ["DO_SKIP_MZN_CHECK"] = "1" 

other useful import

In [ ]:
from discrete_optimization.maximum_independent_set.mis_model import MisProblem
from discrete_optimization.maximum_independent_set.solvers.mis_gurobi import MisMilpSolver
from discrete_optimization.maximum_independent_set.mis_plot import plot_mis_solution, plot_mis_graph
from discrete_optimization.maximum_independent_set.solvers.mis_quantum import QAOAMisSolver
from qiskit_aer import AerSimulator
import networkx as nx

The objectif of this tutorial is to present how we can use discrete_optimization to solve optimization problem using quantum simulator or quantum real device with qiskit. 

It's free to use simulator but to use quantum real divice you need an IBM account. You can have one for free BUT you will be limited to ten minutes of use by month and you can't use session (we talk about session a bit later) who is a very pratical way to execute job on real device.

This tutorial has not to purpose to present a method to solve very large problem, nothing of revolutionnary here, just a quantum gadget to see what we can do actually with quantum technologies, the good point as the bad point.

The first step is, of course, to creat the problem to solve, here we are going to solve a maximum_independent_set.
In a graph we search a subset of nodes who are not connected two by two. We want to maximize the side of this subset.
An example of a graph with 6 nodes where the maximum_independent_set is the subset of nodes (1,5,6).

In [ ]:
    graph = nx.Graph()

    graph.add_edge(1, 2)
    graph.add_edge(1, 3)
    graph.add_edge(2, 4)
    graph.add_edge(2, 6)
    graph.add_edge(3, 4)
    graph.add_edge(3, 5)
    graph.add_edge(4, 5)
    graph.add_edge(4, 6)

    misProblem = MisProblem(graph)
    plot_mis_graph(misProblem)

We are going to see how solve this problem with a non-noisy quantum simulator or a real quantum device. We develop a tools that just need a MisProblem object and a qiskit backend object and who can solve the mis Problem using the QAOA algorithm. Quantum approximate optimization algorithm is a well-known hybrid algorithm that use the Ising representation of a QUBO problem. Article présentant l'algorithm QAOA : https://arxiv.org/abs/1411.4028 

In [ ]:
# we declare the MisProblem
misProblem = MisProblem(graph)

# we declare the QAOA solver we are going to use
misSolver = QAOAMisSolver(misProblem)

# then we need a backend, a backend is an object wo define how and where the quantum part of the algorithm is run
# it can be a simulator or a real quantum device

# here we declare a simulator
backend = AerSimulator()

# then we need to initialize the solver, in fact we transform the mis problem on his QUBO form
misSolver.init_model()

# after that we can run the solver using the defined backend
res = misSolver.solve(backend=backend)

sol, _ = res.get_best_solution_fit()
print(sol)
plot_mis_solution(sol)

If you have an IBM account you can declare the backend as follow, where the "token" parameter is your personnal IBM token to use a real quantum device. If you have a premium account please set the parameter "use-session" of the solver to True. A session give a guaranted acces to the device, if you don't use session, you can have a lot of waiting time beetween two different job (because other jobs can be insert in the queue). In QAOA we create a new job at each iteration of the optimizer and one final job after the minimization step is over.

In [ ]:
# from qiskit_ibm_runtime import QiskitRuntimeService

# service = QiskitRuntimeService(channel="ibm_quantum", token=token)
# backend = service.least_busy(operational=True, simulator=False)
# misSolver.init_model()
# misSolver.solve(backend=backend, use_session=True)
# sol, _ = res.get_best_solution_fit()
# print(sol)

Here we have not define any parameters except the backend, in fact there is a lot of parameters we can chose.

They can be defined in a kwargs parameter

Hyperparameters we can modifiy and can influence the performance of the algorithm are :

- "method" the scipy optimizer we want to use for the classical optimization step
- "reps" : the deep of the quantum circuit
- "optimization_level" : the level of optimization of the circuit
- "nb_shots" who define the number of time we want to repeat the quantum circuit (not used on simulator)

by default we use the "COBYLA" method, you can give value for three hyperparameters in the kwargs :

- maxiter : the maximum number of iteration of the optimizer
- rhobeg : reasonable initial changes to the variables
- tol : tolerence to consider the convergence of the minimization

for all other scipy optimizer https://docs.scipy.org/doc/scipy-1.13.1/reference/generated/scipy.optimize.minimize.html (or it's also possible for cobyla), if you want to define some parameters you can do it with a kwargs parameter "options", who is a dictionnary of possible options for the optimizer.



In [ ]:
# kwargs = {"method": "your_method"}
# options = {"options1": op1, "options2": op2, ...}
# kwargs["options"] = options

There is also the possibility to use an optimizer define in qiskit_algorithms (https://github.com/qiskit-community/qiskit-algorithms/tree/stable/0.3/qiskit_algorithms/optimizers). To use one of them you have just te create it and then passed it in the kwargs as "optimizer" parameter.

In [ ]:
# optimizer = SPSA()
# kwargs = {"optimizer": optimizer} or kwargs["optimizer"] = optimizer if kwargs already exist